<a href="https://colab.research.google.com/github/sarthak-prasad/AudVis_Face_n_Obj/blob/master/YT_playlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pytube

     |████████████████████████████████| 61kB 3.9MB/s 


In [2]:
pip install moviepy

In [3]:
import os
import moviepy.editor as mp
from pytube import YouTube
import string
from google.colab import files
import shutil

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2392064/45929032 bytes (5.2%)5283840/45929032 bytes (11.5%)8093696/45929032 bytes (17.6%)11001856/45929032 bytes (24.0%)13737984/45929032 bytes (29.9%)16580608/45929032 bytes (36.1%)19562496/45929032 bytes (42.6%)22462464/45929032 bytes (48.9%)25346048/45929032 bytes (55.2%)28295168/45929032 bytes (61.6%)31170560/45929032 bytes (67.9%)34029568/45929032 bytes (74.1%)

In [4]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

# setup
gc = gspread.authorize(GoogleCredentials.get_application_default())

# read data and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/10XMuQ6yJPKNKFT21TeaZBqMk5N8lBb8sXcpzhctrG4M/edit?usp=sharing')
sheets = gsheets.worksheet('Form Responses 1').get_all_values()
df = pd.DataFrame(sheets[1:], columns=sheets[0])
df.drop_duplicates(subset="YouTube Link",keep="first",inplace=True)

In [5]:
def createFolder(directory):
  try:
    if not os.path.exists(directory):
      os.makedirs(directory)
  except OSError:
    print ('Error: Creating directory. ' +  directory)

In [6]:
createFolder("/content/Songs")

In [7]:
def flname(a):
  b="p"
  for i in range(len(a)):
    if (a[i] not in ["|",",",'''"''',".",":"]): # Can't allow these characters in filename
      b+=str(a[i])
  return b[1:]

In [8]:
def vid_download(link) :
  yt = YouTube(link)

  title_name =yt.title
  # YT Video details
  print("Title: ",yt.title)
  print("Number of views: ",yt.views)
  print("Length of video: ",yt.length)
  print("Rating of video: ",yt.rating)
  # Getting the highest resolution possible
  ys = yt.streams.get_highest_resolution()

  print("Downloading...")
  ys.download("/content/Songs") # Downloading MP4 file of video
  print("Download completed!!")
  v_loc="/content/Songs/"+flname(title_name)+".mp4"

  return [title_name,v_loc]

In [9]:
singer_key = [ ("KK","a"), ("Shreya Ghoshal","b"), ("Arijit Singh","c"), ("Sonu Nigam",'d'), ("Sunidhi Chauhan","e"), ("Atif Aslam","f"), ("Nusrat Fateh Ali Khan","g"), ("Mohit Suri","h"), ("Neeraj Shridhar","i"), ("Kumar Sanu","j"), ("Alka Yagnik","k"), ("Others","l") ]
category_key =[ ("Sad","a"), ("Romantic","b"), ("Party","c"), ("Qawalli","d"), ("Calm","e"), ("Others","f") ]

# Extracting Audio from Video 

def vidtoaud(title_name,v_loc,category,singer):
  a_loc="/content/Songs/"+flname(title_name)+ ".mp3"
  my_clip = mp.VideoFileClip(v_loc)
  print(my_clip)
  my_clip.audio.write_audiofile(a_loc)
  os.remove(v_loc)

In [10]:
# User Input

choice = int (input("Press 0 to choose by category , 1 to choose by singer and 2 to get Lord Emraan's playlist "))
print("\n")

if (choice==1) :
  for x,y in singer_key:
    print(y + " : " + x + "\n")
elif (choice==0) :
  for x,y in category_key:
    print(y + " : " + x + "\n")

if (choice==0 or choice==1) :
  ref = str (input("Write reference ID "))
  print("\n")
  ref_id = string.ascii_lowercase.index(ref)

if (choice==1) :
  v = singer_key[ref_id][0]
  print("Songs by "+v+" on their way !")
elif (choice==0):
  v = category_key[ref_id][0]
  print(v+" songs on their way !")
elif (choice ==2) :
  v = "Lord Emraan"
  print("Lord Emraan's playlist on its way !")
else :
  print("Invalid Input")

Press 0 to choose by category , 1 to choose by singer and 2 to get Lord Emraan's playlist 1


a : KK

b : Shreya Ghoshal

c : Arijit Singh

d : Sonu Nigam

e : Sunidhi Chauhan

f : Atif Aslam

g : Nusrat Fateh Ali Khan

h : Mohit Suri

i : Neeraj Shridhar

j : Kumar Sanu

k : Alka Yagnik

l : Others

Write reference ID f


Songs by Atif Aslam on their way !


In [11]:
def print_playlist(count_index,s,c,l) :
  l = YouTube(l)
 # title_name = l.title
  print(str (count_index) + " : " + l.title ) 
  print( "--- || " + s + " --- " + c + " || ---" )
  print("\n")


browse_ch = int (input("Wish to browse through your playlist ? Press 0 "+"\n"+"Wish to browse through the entire collection ? Press 1 \n"))
print("\n")

song_no = 0
count_index = 1
num_entries = len(df.index)

if (browse_ch == 0) :
  while (song_no < num_entries):
    try :
      c = df.at[song_no, "Category"]
      s = df.at[song_no, "Sort by Singer"]
      l = df.at[song_no, "YouTube Link"]
      e = df.at[song_no, "Hey, Is it an Emraan Hashmi song ?"]

      if (choice==1):
        if s.find(v)!=-1 :
          print_playlist(count_index,s,c,l)
          count_index += 1
      elif (choice==0):
        if c.find(v)!=-1 :
          print_playlist(count_index,s,c,l)
          count_index += 1
      elif (choice==2) :
        if e == "Yup !!!" :
          print_playlist(count_index,s,c,l)
          count_index += 1
      
    except Exception as expn : 
      print(expn)
      
    song_no += 1

elif (browse_ch == 1) :
  while (song_no < num_entries):
    try :
      c = df.at[song_no, "Category"]
      s = df.at[song_no, "Sort by Singer"]
      l = df.at[song_no, "YouTube Link"]
      e = df.at[song_no, "Hey, Is it an Emraan Hashmi song ?"]
      print_playlist(count_index,s,c,l)
      count_index += 1
    except Exception as expn : 
      print(expn)
      
    song_no += 1

else :
  print("Invalid Input")

if (browse_ch == 1 or browse_ch == 0) and (count_index == 1) :
  print("No such Songs are available in our collection ")

Wish to browse through your playlist ? Press 0 
Wish to browse through the entire collection ? Press 1 
0


1 : Aadat (Juda Hoke Bhi) - Atif Aslam - Kunal Khemu - Kalyug [2005]
--- || Atif Aslam --- Sad, Calm || ---


2 : Woh Lamhe Woh Baatein | Audio | Atif Aslam | Emraan Hashmi | Zeher | Shamita Shetty | Udita Goswami
--- || Atif Aslam --- Romantic, Calm || ---


3 : Jeena Jeena (Audio Song) | Badlapur | Varun Dhawan, Yami Gautam & Nawazuddin Siddiqui
--- || Atif Aslam --- Romantic, Calm || ---




In [12]:
song_no = 0
count_index = 0
num_entries = len(df.index)
while (song_no < num_entries):
  try :
    c = df.at[song_no, "Category"]
    s = df.at[song_no, "Sort by Singer"]
    l = df.at[song_no, "YouTube Link"]
    e = df.at[song_no, "Hey, Is it an Emraan Hashmi song ?"]

    if (choice==1):
      if s.find(v)!=-1 :
        count_index += 1
        a,b = vid_download(l)
        vidtoaud(a,b,c,s)
    elif (choice==0):
      if c.find(v)!=-1 :
        count_index += 1
        a,b = vid_download(l)
        vidtoaud(a,b,c,s)
    elif (choice==2) :
      if e == "Yup !!!" :
        count_index += 1
        a,b = vid_download(l)
        vidtoaud(a,b,c,s)
    
  except Exception as expn : 
    print(expn)
    print("\n")
    
  song_no += 1

if (count_index == 0) :
  print("No such Songs are available in our collection ")
else :
  print("Playlist Getting Ready !!!")

  # Creating the zip file

  old_name = "/content/Songs"
  new_name = "/content/" + v
  os.rename(old_name,new_name)
  shutil.make_archive(new_name, 'zip', new_name)
  new_name += ".zip"
  files.download(new_name)

Title:  Aadat (Juda Hoke Bhi) - Atif Aslam - Kunal Khemu - Kalyug [2005]
Number of views:  31786550
Length of video:  335
Rating of video:  4.8183942
Downloading...
Download completed!!
[MoviePy] Writing audio in /content/Songs/Aadat (Juda Hoke Bhi) - Atif Aslam - Kunal Khemu - Kalyug [2005].mp3


100%|██████████| 7389/7389 [00:11<00:00, 649.47it/s]

[MoviePy] Done.


Title:  Woh Lamhe Woh Baatein | Audio | Atif Aslam | Emraan Hashmi | Zeher | Shamita Shetty | Udita Goswami
Number of views:  45681462
Length of video:  320
Rating of video:  4.8088603
Downloading...
Download completed!!
[MoviePy] Writing audio in /content/Songs/Woh Lamhe Woh Baatein  Audio  Atif Aslam  Emraan Hashmi  Zeher  Shamita Shetty  Udita Goswami.mp3


100%|██████████| 7058/7058 [00:11<00:00, 620.70it/s]

...

100%|██████████| 5186/5186 [00:08<00:00, 604.57it/s]

[MoviePy] Done.


Playlist Getting Ready !!!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# Clearing the Folders and Files
dir = "/content/" + v
for f in os.listdir(dir):
  os.remove(os.path.join(dir, f))
os.rmdir(dir) 
os.remove(new_name)